# VacationPy

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
file_path = 'output_data/cities.csv'
weather_data = pd.read_csv(file_path)
weather_data

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Chatra,0,IN,1586732317,73,24.22,84.87,75.20,3.22
1,Mataura,73,NZ,1586731493,81,-46.19,168.86,45.00,14.00
2,Longyearbyen,40,SJ,1586731995,70,78.22,15.64,-4.00,2.24
3,Champerico,49,GT,1586732328,65,14.30,-91.92,85.50,7.85
4,Provideniya,90,RU,1586731758,80,64.38,-173.30,30.20,11.18
...,...,...,...,...,...,...,...,...,...
537,Orlik,85,RU,1586732683,61,52.52,99.83,25.70,2.51
538,Daru,32,PG,1586732683,72,-9.08,143.21,84.24,8.63
539,Ouadda,97,CF,1586732684,39,8.08,22.40,69.55,1.03
540,Soyo,99,AO,1586732684,89,-6.13,12.37,77.16,7.65


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
#Store the maximum humidity value observed in the data set
max_hum = weather_data['Humidity'].max()
max_hum

100

In [6]:
#Create a heat map of all cities in the dataset using humidity as the weight
#Configure gmaps with api key
gmaps.configure(api_key=g_key)

#Store the lat/long values for each city
locations = weather_data[['Lat', 'Lng']].astype(float)

#Store the humidity values for each city
humidity = weather_data['Humidity'].astype(float)


# Create a heatmap of humidity for all cities in the data frame
fig = gmaps.figure()

#Create the heat map layer
hum_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max_hum,
                                 point_radius = 1)

#Add the heat map layer to the figure
fig.add_layer(hum_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [9]:
#Narrow down the list of cities to those with wind speed below 10 mph
ideal_winds = weather_data.loc[weather_data['Wind Speed'] < 10]

#Narrow the list of cities to those with max temperatures below 80 degrees Fahrenheit
upper_temps = ideal_winds.loc[ideal_winds['Max Temp'] < 80]

#Narrow the list of cities to those with max temperatures above 70 degrees
lower_temps = upper_temps.loc[upper_temps['Max Temp'] > 70]

#Narrow the list of cities to those with 0 percent cloud cover
ideal_cities = lower_temps.loc[lower_temps['Cloudiness'] == 0].reset_index(drop=True)

#View the data frame of cities that match all of our criteria
ideal_cities

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Chatra,0,IN,1586732317,73,24.22,84.87,75.20,3.22
1,Dhing,0,IN,1586732416,45,26.47,92.47,71.82,4.18
2,Corumba,0,BR,1586732043,59,-19.01,-57.65,75.97,1.48
3,Ladario,0,BR,1586732582,59,-19.00,-57.60,76.75,1.59
4,Yanam,0,IN,1586732593,82,16.73,82.22,78.17,3.38
5,Arani,0,IN,1586732637,70,12.67,79.28,74.62,2.53


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [11]:
#Create a copy of the ideal cities data frame
hotel_df = ideal_cities
#Add an empty column for Hotel Name
hotel_df['Hotel Name'] = ''

#Verify the Hotel Name column was added correctly
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,Chatra,0,IN,1586732317,73,24.22,84.87,75.20,3.22,
1,Dhing,0,IN,1586732416,45,26.47,92.47,71.82,4.18,
2,Corumba,0,BR,1586732043,59,-19.01,-57.65,75.97,1.48,
3,Ladario,0,BR,1586732582,59,-19.00,-57.60,76.75,1.59,
4,Yanam,0,IN,1586732593,82,16.73,82.22,78.17,3.38,
5,Arani,0,IN,1586732637,70,12.67,79.28,74.62,2.53,


In [14]:
#Base url for API request
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

#Iterate through each city in the data frame
for index, row in hotel_df.iterrows():
    
    #Variables for search parameters
    coordinates = f"{row['Lat']}, {row['Lng']}"
    search_radius = 5000
    place_type = 'lodging'
    
    #Search parameters
    params = {
        'location': coordinates,
        'radius': search_radius,
        'type': place_type,
        'key': g_key
    }
    
    #Make API request
    response = requests.get(url, params=params).json()
    
    #Store the first result and append
    first_hotel = response['results'][0]
    #Add the hotel name to the appropriate column in the hotel data frame
    hotel_df.loc[index, 'Hotel Name'] = first_hotel['name']

In [15]:
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,Chatra,0,IN,1586732317,73,24.22,84.87,75.20,3.22,Kushwanshi bhawan
1,Dhing,0,IN,1586732416,45,26.47,92.47,71.82,4.18,Barua Restaurent
2,Corumba,0,BR,1586732043,59,-19.01,-57.65,75.97,1.48,Hotel Laura Vicunha
3,Ladario,0,BR,1586732582,59,-19.00,-57.60,76.75,1.59,Hotel Fazenda 4 cantos
4,Yanam,0,IN,1586732593,82,16.73,82.22,78.17,3.38,The Regency Hotel
5,Arani,0,IN,1586732637,70,12.67,79.28,74.62,2.53,KPK LODGE


In [17]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [18]:
 # Create hotel symbol layer
hotel_layer = gmaps.marker_layer(
    locations,
    info_box_content=hotel_info)

#Create a combined map of the heat map and hotel symbol layers
fig = gmaps.figure()

#Add the heat map and hotel symbol layers
fig.add_layer(hum_layer)
fig.add_layer(hotel_layer)

#Display the combined map
fig

Figure(layout=FigureLayout(height='420px'))